# thoughts

# data read 

In [1]:
import glob
import re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection m

In [2]:
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/train.csv')

IOError: File ./input/train.csv does not exist

# basic func

In [3]:
dates={}
def lookup(s):
    global dates
    tmp_dates={}
    """
    This is an extremely fast approach to datetime parsing.
    For large data, the same dates are often repeated. Rather than
    re-parse these, we store all unique dates, parse them, and
    use a lookup to convert all dates.
    """
    for date in s.unique():
        if date in dates:
            tmp_dates[date] = dates[date]
        else:
            dates[date]=pd.to_datetime(date)
            tmp_dates[date]=dates[date]
            
                
#         dates[date] = dates[date] if date in dates else pd.to_datetime(date)
#     dates = {date: pd.to_datetime(date) for date in s.unique()}
    return s.map(dates)

In [4]:
import pandas as pd
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

In [19]:
import time

# rnn 

In [13]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dropout, Dense, concatenate, GRU, Embedding, Flatten, Activation
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K

Using TensorFlow backend.


In [14]:
def rmsle(Y, Y_pred):
    # Y and Y_red have already been in log scale.
    assert Y.shape == Y_pred.shape
    return np.sqrt(np.mean(np.square(Y_pred - Y )))

In [15]:
%%time

train_df = pd.read_table('../input/train.tsv')
test_df = pd.read_table('../input/test.tsv')
print(train_df.shape, test_df.shape)

((1482535, 8), (693359, 7))
CPU times: user 7.37 s, sys: 706 ms, total: 8.07 s
Wall time: 8.11 s


In [16]:
def fill_missing_values(df):
    df.category_name.fillna(value="Other", inplace=True)
    df.brand_name.fillna(value="missing", inplace=True)
    df.item_description.fillna(value="None", inplace=True)
    return df

train_df = fill_missing_values(train_df)
test_df = fill_missing_values(test_df)

In [17]:
def fill_missing_values(df):
    df.category_name.fillna(value="Other", inplace=True)
    df.brand_name.fillna(value="missing", inplace=True)
    df.item_description.fillna(value="None", inplace=True)
    return df

train_df = fill_missing_values(train_df)
test_df = fill_missing_values(test_df)
# Scale target variable to log.
train_df["target"] = np.log1p(train_df.price)

# Split training examples into train/dev examples.
train_df, dev_df = train_test_split(train_df, random_state=347, train_size=0.99)

Y_train = train_df.target.values.reshape(-11, 1)
Y_dev = dev_df.target.values.reshape(-1, 1)

# Calculate number of train/dev/test examples.
n_trains = train_df.shape[0]
n_devs = dev_df.shape[0]
n_tests = test_df.shape[0]
print("Training on", n_trains, "examples")
print("Validating on", n_devs, "examples")
print("Testing on", n_tests, "examples")

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


('Training on', 1467709, 'examples')
('Validating on', 14826, 'examples')
('Testing on', 693359, 'examples')


In [18]:
full_df = pd.concat([train_df, dev_df, test_df])

In [19]:
%%time

print("Processing categorical data...")
le = LabelEncoder()

le.fit(full_df.category_name)
full_df.category_name = le.transform(full_df.category_name)

le.fit(full_df.brand_name)
full_df.brand_name = le.transform(full_df.brand_name)

del le

Processing categorical data...
CPU times: user 11.7 s, sys: 65.3 ms, total: 11.7 s
Wall time: 11.7 s


In [20]:
%%time

print("Transforming text data to sequences...")
raw_text = np.hstack([full_df.item_description.str.lower(), full_df.name.str.lower()])

print("   Fitting tokenizer...")
tok_raw = Tokenizer()
tok_raw.fit_on_texts(raw_text)

print("   Transforming text to sequences...")
full_df['seq_item_description'] = tok_raw.texts_to_sequences(full_df.item_description.str.lower())
full_df['seq_name'] = tok_raw.texts_to_sequences(full_df.name.str.lower())

del tok_raw

Transforming text data to sequences...
   Fitting tokenizer...
   Transforming text to sequences...
CPU times: user 2min 59s, sys: 1.9 s, total: 3min 1s
Wall time: 3min


In [21]:
# Define constants to use when define RNN model
MAX_NAME_SEQ = 10
MAX_ITEM_DESC_SEQ = 75
MAX_TEXT = np.max([
    np.max(full_df.seq_name.max()),
    np.max(full_df.seq_item_description.max()),
]) + 4
MAX_CATEGORY = np.max(full_df.category_name.max()) + 1
MAX_BRAND = np.max(full_df.brand_name.max()) + 1
MAX_CONDITION = np.max(full_df.item_condition_id.max()) + 1

In [22]:
%%time

def get_keras_data(df):
    X = {
        'name': pad_sequences(df.seq_name, maxlen=MAX_NAME_SEQ),
        'item_desc': pad_sequences(df.seq_item_description, maxlen=MAX_ITEM_DESC_SEQ),
        'brand_name': np.array(df.brand_name),
        'category_name': np.array(df.category_name),
        'item_condition': np.array(df.item_condition_id),
        'num_vars': np.array(df[["shipping"]]),
    }
    return X

train = full_df[:n_trains]
dev = full_df[n_trains:n_trains+n_devs]
test = full_df[n_trains+n_devs:]

X_train = get_keras_data(train)
X_dev = get_keras_data(dev)
X_test = get_keras_data(test)

CPU times: user 19.5 s, sys: 1.62 s, total: 21.1 s
Wall time: 21.9 s


In [23]:
def new_rnn_model(lr=0.001, decay=0.0):    
    # Inputs
    name = Input(shape=[X_train["name"].shape[1]], name="name")
    item_desc = Input(shape=[X_train["item_desc"].shape[1]], name="item_desc")
    brand_name = Input(shape=[1], name="brand_name")
    category_name = Input(shape=[1], name="category_name")
    item_condition = Input(shape=[1], name="item_condition")
    num_vars = Input(shape=[X_train["num_vars"].shape[1]], name="num_vars")

    # Embeddings layers
    emb_name = Embedding(MAX_TEXT, 20)(name)
    emb_item_desc = Embedding(MAX_TEXT, 60)(item_desc)
    emb_brand_name = Embedding(MAX_BRAND, 10)(brand_name)
    emb_category_name = Embedding(MAX_CATEGORY, 10)(category_name)
    emb_item_condition = Embedding(MAX_CONDITION, 5)(item_condition)

    # rnn layers
    rnn_layer1 = GRU(16) (emb_item_desc)
    rnn_layer2 = GRU(8) (emb_name)

    # main layers
    main_l = concatenate([
        Flatten() (emb_brand_name)
        , Flatten() (emb_category_name)
        , Flatten() (emb_item_condition)
        , rnn_layer1
        , rnn_layer2
        , num_vars
    ])

    main_l = Dense(256)(main_l)
    main_l = Activation('elu')(main_l)

    main_l = Dense(128)(main_l)
    main_l = Activation('elu')(main_l)

    main_l = Dense(64)(main_l)
    main_l = Activation('elu')(main_l)

    # the output layer.
    output = Dense(1, activation="linear") (main_l)

    model = Model([name, item_desc, brand_name , category_name, item_condition, num_vars], output)

    optimizer = Adam(lr=lr, decay=decay)
    model.compile(loss="mse", optimizer=optimizer)

    return model

model = new_rnn_model()
model.summary()
del model

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
brand_name (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
category_name (InputLayer)      (None, 1)            0                                            
__________________________________________________________________________________________________
item_condition (InputLayer)     (None, 1)            0                                            
__________________________________________________________________________________________________
item_desc (InputLayer)          (None, 75)           0                                            
__________________________________________________________________________________________________
name (Inpu

In [24]:
# Set hyper parameters for the model.
BATCH_SIZE = 1024
epochs = 2

# Calculate learning rate decay.
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
steps = int(n_trains / BATCH_SIZE) * epochs
lr_init, lr_fin = 0.007, 0.0005
lr_decay = exp_decay(lr_init, lr_fin, steps)

rnn_model = new_rnn_model(lr=lr_init, decay=lr_decay)

print("Fitting RNN model to training examples...")
rnn_model.fit(
        X_train, Y_train, epochs=epochs, batch_size=BATCH_SIZE,
        validation_data=(X_dev, Y_dev), verbose=2,
)

Fitting RNN model to training examples...
Train on 1467709 samples, validate on 14826 samples
Epoch 1/2
 - 706s - loss: 0.2428 - val_loss: 0.1964
Epoch 2/2
 - 660s - loss: 0.1704 - val_loss: 0.1869


In [25]:
%%time

print("Evaluating the model on validation data...")
Y_dev_preds_rnn = rnn_model.predict(X_dev, batch_size=BATCH_SIZE)
print(" RMSLE error:", rmsle(Y_dev, Y_dev_preds_rnn))

Evaluating the model on validation data...
(' RMSLE error:', 0.43236985495311914)
CPU times: user 4.24 s, sys: 899 ms, total: 5.14 s
Wall time: 1.27 s


In [26]:
rnn_preds = rnn_model.predict(X_test, batch_size=BATCH_SIZE, verbose=1)
rnn_preds = np.expm1(rnn_preds)

693359/693359 [==============================] - 53s 76us/step


In [27]:
full_df = pd.concat([train_df, dev_df, test_df])

In [28]:
%%time

# Convert data type to string
full_df['shipping'] = full_df['shipping'].astype(str)
full_df['item_condition_id'] = full_df['item_condition_id'].astype(str)

CPU times: user 2.17 s, sys: 30.1 ms, total: 2.2 s
Wall time: 2.21 s


In [29]:
%%time

print("Vectorizing data...")
default_preprocessor = CountVectorizer().build_preprocessor()
def build_preprocessor(field):
    field_idx = list(full_df.columns).index(field)
    return lambda x: default_preprocessor(x[field_idx])

vectorizer = FeatureUnion([
    ('name', CountVectorizer(
        ngram_range=(1, 2),
        max_features=50000,
        preprocessor=build_preprocessor('name'))),
    ('category_name', CountVectorizer(
        token_pattern='.+',
        preprocessor=build_preprocessor('category_name'))),
    ('brand_name', CountVectorizer(
        token_pattern='.+',
        preprocessor=build_preprocessor('brand_name'))),
    ('shipping', CountVectorizer(
        token_pattern='\d+',
        preprocessor=build_preprocessor('shipping'))),
    ('item_condition_id', CountVectorizer(
        token_pattern='\d+',
        preprocessor=build_preprocessor('item_condition_id'))),
    ('item_description', TfidfVectorizer(
        ngram_range=(1, 3),
        max_features=100000,
        preprocessor=build_preprocessor('item_description'))),
])

X = vectorizer.fit_transform(full_df.values)

X_train = X[:n_trains]
X_dev = X[n_trains:n_trains+n_devs]
X_test = X[n_trains+n_devs:]

print(X.shape, X_train.shape, X_dev.shape, X_test.shape)

Vectorizing data...


UnicodeDecodeError: 'ascii' codec can't decode byte 0xe2 in position 5: ordinal not in range(128)

# lgbm

In [7]:
import gc
import time
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import lightgbm as lgb

import sys

#Add https://www.kaggle.com/anttip/wordbatch to your kernel Data Sources, 
#until Kaggle admins fix the wordbatch pip package installation
# sys.path.insert(0, '../input/wordbatch/')#origin ../input/wordbatch/wordbatch/
import wordbatch 

from wordbatch.extractors import WordBag, WordHash
from wordbatch.models import FTRL, FM_FTRL

from nltk.corpus import stopwords
import re

NUM_BRANDS = 4500
NUM_CATEGORIES = 1250

# develop = False
develop= True

def rmsle(y, y0):
    assert len(y) == len(y0)
    return np.sqrt(np.mean(np.power(np.log1p(y) - np.log1p(y0), 2)))


def split_cat(text):
    try:
        return text.split("/")
    except:
        return ("No Label", "No Label", "No Label")


def handle_missing_inplace(dataset):
    dataset['general_cat'].fillna(value='missing', inplace=True)
    dataset['subcat_1'].fillna(value='missing', inplace=True)
    dataset['subcat_2'].fillna(value='missing', inplace=True)
    dataset['brand_name'].fillna(value='missing', inplace=True)
    dataset['item_description'].fillna(value='missing', inplace=True)


def cutting(dataset):
    pop_brand = dataset['brand_name'].value_counts().loc[lambda x: x.index != 'missing'].index[:NUM_BRANDS]
    dataset.loc[~dataset['brand_name'].isin(pop_brand), 'brand_name'] = 'missing'
    pop_category1 = dataset['general_cat'].value_counts().loc[lambda x: x.index != 'missing'].index[:NUM_CATEGORIES]
    pop_category2 = dataset['subcat_1'].value_counts().loc[lambda x: x.index != 'missing'].index[:NUM_CATEGORIES]
    pop_category3 = dataset['subcat_2'].value_counts().loc[lambda x: x.index != 'missing'].index[:NUM_CATEGORIES]
    dataset.loc[~dataset['general_cat'].isin(pop_category1), 'general_cat'] = 'missing'
    dataset.loc[~dataset['subcat_1'].isin(pop_category2), 'subcat_1'] = 'missing'
    dataset.loc[~dataset['subcat_2'].isin(pop_category3), 'subcat_2'] = 'missing'


def to_categorical(dataset):
    dataset['general_cat'] = dataset['general_cat'].astype('category')
    dataset['subcat_1'] = dataset['subcat_1'].astype('category')
    dataset['subcat_2'] = dataset['subcat_2'].astype('category')
    dataset['item_condition_id'] = dataset['item_condition_id'].astype('category')


# Define helpers for text normalization
stopwords = {x: 1 for x in stopwords.words('english')}
non_alphanums = re.compile(u'[^A-Za-z0-9]+')


def normalize_text(text):
    return u" ".join(
        [x for x in [y for y in non_alphanums.sub(' ', text).lower().strip().split(" ")] \
         if len(x) > 1 and x not in stopwords])

In [41]:
train.head()

,brand_name,category_name,item_condition_id,item_description,name,price,shipping,target,test_id,train_id,seq_item_description,seq_name
600141,4588,503,2,Super cute! No flaws! Non smoking home!,Nurse Decoration,10.0,1,2.397895,NaN,600141.0,"[98, 90, 12, 217, 569, 1377, 102]","[5729, 2551]"
844745,3337,593,3,Size 9c,Jordan 12 All Blue,46.0,0,3.850148,NaN,844745.0,"[6, 4611]","[412, 147, 35, 66]"
1055092,1160,523,3,Corning Ware 6 cup tea pot with stainless lid....,Corning Ware Tea Pot,16.0,0,2.833213,NaN,1055092.0,"[13131, 4828, 45, 487, 1047, 2486, 9, 700, 140...","[13131, 4828, 1047, 2486]"
121713,5265,1162,2,No description yet,Bundle for Morefish,27.0,0,3.332205,NaN,121713.0,"[12, 68, 79]","[27, 3, 265114]"
1159852,4953,1217,2,3.4 no box,Victoria secret very sexy,27.0,0,3.332205,NaN,1159852.0,"[32, 47, 12, 57]","[231, 81, 67, 426]"


In [42]:
from time import gmtime, strftime
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

# if 1 == 1:
train = pd.read_table(
    '../input/train.tsv', engine='c')
test = pd.read_table(
    '../input/test.tsv', engine='c')


2018-02-08 09:05:55


In [45]:
train.head(100)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity
5,5,Bundled items requested for Ruie,3,Women/Other/Other,NaN,59.0,0,"Banana republic bottoms, Candies skirt with ma..."
6,6,Acacia pacific tides santorini top,3,Women/Swimwear/Two-Piece,Acacia Swimwear,64.0,0,Size small but straps slightly shortened to fi...
7,7,Girls cheer and tumbling bundle of 7,3,Sports & Outdoors/Apparel/Girls,Soffe,6.0,1,You get three pairs of Sophie cheer shorts siz...
8,8,Girls Nike Pro shorts,3,Sports & Outdoors/Apparel/Girls,Nike,19.0,0,Girls Size small Plus green. Three shorts total.
9,9,Porcelain clown doll checker pants VTG,3,Vintage & Collectibles/Collectibles/Doll,NaN,8.0,0,I realized his pants are on backwards after th...


In [6]:
start_time = time.time()

#train = pd.read_table('../input/train.tsv', engine='c')
#test = pd.read_table('../input/test.tsv', engine='c')

print('[{}] Finished to load data'.format(time.time() - start_time))
print('Train shape: ', train.shape)
print('Test shape: ', test.shape)
nrow_test = train.shape[0]  # -dftt.shape[0]
dftt = train[(train.price < 1.0)]
train = train.drop(train[(train.price < 1.0)].index)
del dftt['price']
nrow_train = train.shape[0]
# print(nrow_train, nrow_test)
y = np.log1p(train["price"])
merge= pd.concat([train, dftt, test])
submission = test[['test_id']]

del train
del test
gc.collect()

merge['general_cat'], merge['subcat_1'], merge['subcat_2'] = \
    zip(*merge['category_name'].apply(lambda x: split_cat(x)))
merge.drop('category_name', axis=1, inplace=True)
print('[{}] Split categories completed.'.format(time.time() - start_time))

handle_missing_inplace(merge)
print('[{}] Handle missing completed.'.format(time.time() - start_time))

cutting(merge)
print('[{}] Cut completed.'.format(time.time() - start_time))

to_categorical(merge)
print('[{}] Convert categorical completed'.format(time.time() - start_time))

wb = wordbatch.WordBatch(normalize_text, extractor=(WordBag, {"hash_ngrams": 2, "hash_ngrams_weights": [1.5, 1.0],
                                                              "hash_size": 2 ** 22, "norm": None, "tf": 'binary',
                                                              "idf": None,
                                                              }), procs=8)
wb.dictionary_freeze = True
X_name = wb.fit_transform(merge['name'])
del(wb)
X_name = X_name[:, np.array(
    np.clip(X_name.getnnz(axis=0) - 1, 0, 1), dtype=bool)]
print('[{}] Vectorize `name` completed.'.format(time.time() - start_time))

wb = CountVectorizer()
X_category1 = wb.fit_transform(merge['general_cat'])
X_category2 = wb.fit_transform(merge['subcat_1'])
X_category3 = wb.fit_transform(merge['subcat_2'])
print('[{}] Count vectorize `categories` completed.'.format(
    time.time() - start_time))

# wb= wordbatch.WordBatch(normalize_text, extractor=(WordBag, {"hash_ngrams": 3, "hash_ngrams_weights": [1.0, 1.0, 0.5],




2018-02-08 03:04:35
[7.97957992554] Finished to load data
('Train shape: ', (1482535, 8))
('Test shape: ', (693359, 7))
[15.7959108353] Split categories completed.
[16.6497588158] Handle missing completed.
[19.0920028687] Cut completed.
[20.522397995] Convert categorical completed
Normalize text
Extract wordbags
[143.54538393] Vectorize `name` completed.
[180.01602602] Count vectorize `categories` completed.
Normalize text
Extract wordbags
[400.667779922] Vectorize `item_description` completed.


KeyboardInterrupt: 

In [39]:
train.head()

,brand_name,category_name,item_condition_id,item_description,name,price,shipping,target,test_id,train_id,seq_item_description,seq_name
600141,4588,503,2,Super cute! No flaws! Non smoking home!,Nurse Decoration,10.0,1,2.397895,NaN,600141.0,"[98, 90, 12, 217, 569, 1377, 102]","[5729, 2551]"
844745,3337,593,3,Size 9c,Jordan 12 All Blue,46.0,0,3.850148,NaN,844745.0,"[6, 4611]","[412, 147, 35, 66]"
1055092,1160,523,3,Corning Ware 6 cup tea pot with stainless lid....,Corning Ware Tea Pot,16.0,0,2.833213,NaN,1055092.0,"[13131, 4828, 45, 487, 1047, 2486, 9, 700, 140...","[13131, 4828, 1047, 2486]"
121713,5265,1162,2,No description yet,Bundle for Morefish,27.0,0,3.332205,NaN,121713.0,"[12, 68, 79]","[27, 3, 265114]"
1159852,4953,1217,2,3.4 no box,Victoria secret very sexy,27.0,0,3.332205,NaN,1159852.0,"[32, 47, 12, 57]","[231, 81, 67, 426]"


In [ ]:
wb = wordbatch.WordBatch(normalize_text, extractor=(WordBag, {"hash_ngrams": 2, "hash_ngrams_weights": [1.0, 1.0],
                                                              "hash_size": 2 ** 22, "norm": "l2", "tf": 1.0,
                                                              "idf": None}), procs=8)
wb.dictionary_freeze = True
X_description = wb.fit_transform(merge['item_description'])
del(wb)
X_description = X_description[:, np.array(
    np.clip(X_description.getnnz(axis=0) - 1, 0, 1), dtype=bool)]
print('[{}] Vectorize `item_description` completed.'.format(
    time.time() - start_time))

In [9]:
print "???/"

???/


In [10]:

lb = LabelBinarizer(sparse_output=True)
X_brand = lb.fit_transform(merge['brand_name'])
print('[{}] Label binarize `brand_name` completed.'.format(
    time.time() - start_time))

X_dummies = csr_matrix(pd.get_dummies(merge[['item_condition_id', 'shipping']],
                                      sparse=True).values)
print('[{}] Get dummies on `item_condition_id` and `shipping` completed.'.format(
    time.time() - start_time))
print(X_dummies.shape, X_description.shape, X_brand.shape, X_category1.shape, X_category2.shape, X_category3.shape,
      X_name.shape)
sparse_merge = hstack((X_dummies, X_description, X_brand,
                       X_category1, X_category2, X_category3, X_name)).tocsr()

print('[{}] Create sparse merge completed'.format(time.time() - start_time))

#    pd.to_pickle((sparse_merge, y), "xy.pkl")
# else:
#    nrow_train, nrow_test= 1481661, 1482535
#    sparse_merge, y = pd.read_pickle("xy.pkl")

# Remove features with document frequency <=1
print(sparse_merge.shape)
mask = np.array(np.clip(sparse_merge.getnnz(axis=0) - 1, 0, 1), dtype=bool)
sparse_merge = sparse_merge[:, mask]
X = sparse_merge[:nrow_train]
X_test = sparse_merge[nrow_test:]
print(sparse_merge.shape)

[750.549966812] Label binarize `brand_name` completed.
[755.901980877] Get dummies on `item_condition_id` and `shipping` completed.
((2175894, 6), (2175894, 2092352), (2175894, 4501), (2175894, 14), (2175894, 143), (2175894, 977), (2175894, 580342))
[765.825722933] Create sparse merge completed
(2175894, 2678335)
(2175894, 2677819)


In [ ]:
gc.collect()
train_X, train_y = X, y
if develop:
    train_X, valid_X, train_y, valid_y = train_test_split(
        X, y, test_size=0.05, random_state=100)

model = FTRL(alpha=0.01, beta=0.1, L1=0.00001, L2=1.0,
             D=sparse_merge.shape[1], iters=50, inv_link="identity", threads=1)

model.fit(train_X, train_y)
print('[{}] Train FTRL completed'.format(time.time() - start_time))
if develop:
    preds = model.predict(X=valid_X)
    print("FTRL dev RMSLE:", rmsle(np.expm1(valid_y), np.expm1(preds)))

predsF = model.predict(X_test)
print('[{}] Predict FTRL completed'.format(time.time() - start_time))

model = FM_FTRL(alpha=0.01, beta=0.01, L1=0.00001, L2=0.1, D=sparse_merge.shape[1], alpha_fm=0.01, L2_fm=0.0, init_fm=0.01,
                D_fm=200, e_noise=0.0001, iters=17, inv_link="identity", threads=4)

model.fit(train_X, train_y)
print('[{}] Train ridge v2 completed'.format(time.time() - start_time))
if develop:
    preds = model.predict(X=valid_X)
    print("FM_FTRL dev RMSLE:", rmsle(np.expm1(valid_y), np.expm1(preds)))

predsFM = model.predict(X_test)
print('[{}] Predict FM_FTRL completed'.format(time.time() - start_time))


In [38]:

params = {
    'learning_rate': 0.3,
    'application': 'regression',
    'max_depth': 50,
    'num_leaves': 2**10-1,
    'verbosity': -1,
    'metric': 'RMSE',
    'data_random_seed': 1,
    'booster': 'gbtree',
#     'bagging_fraction': 0.8,
#     'bagging_freq': 5,
    'feature_fraction': 0.8,
    'nthread': 4,
    'min_data_in_leaf': 50,
#     'max_bin': 31
}

# Remove features with document frequency <=100
print(sparse_merge.shape)
mask = np.array(np.clip(sparse_merge.getnnz(
    axis=0) - 100, 0, 1), dtype=bool)
sparse_merge = sparse_merge[:, mask]
X = sparse_merge[:nrow_train]
X_test = sparse_merge[nrow_test:]
print(sparse_merge.shape)

train_X, train_y = X, y
if develop:
    train_X, valid_X, train_y, valid_y = train_test_split(
        X, y, test_size=0.05, random_state=100)

d_train = lgb.Dataset(train_X, label=train_y)
watchlist = [d_train]
if develop:
    d_valid = lgb.Dataset(valid_X, label=valid_y)
    watchlist = [d_train, d_valid]

model = lgb.train(params, train_set=d_train, num_boost_round=4000, valid_sets=watchlist,
                  early_stopping_rounds=100, verbose_eval=10)

if develop:
    preds = model.predict(valid_X)
    print("LGB dev RMSLE:", rmsle(np.expm1(valid_y), np.expm1(preds)))

predsL = model.predict(X_test)

print('[{}] Predict LGB completed.'.format(time.time() - start_time))

# preds = (predsF * 0.1 + predsL * 0.1 + predsFM * 0.8)

submission['price'] = np.expm1(preds)
submission.to_csv("submission_wordbatch_ftrl_fm_lgb.csv", index=False)

(2175894, 67816)


KeyboardInterrupt: 

# test

In [345]:
from time import gmtime, strftime
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

# if 1 == 1:
def log1p(x):
    return np.log1p(float(x))

train = pd.read_csv(
    '../input/train.tsv',sep='\t',converters = {"price": log1p,"category_name":unicode,'brand_name':unicode,'item_description':unicode},encoding = 'utf8')
test = pd.read_csv(
    '../input/test.tsv',sep='\t',converters = {"price": log1p,"category_name":unicode,'brand_name':unicode,'item_description':unicode},encoding = 'utf8')

2018-02-11 04:05:16


In [346]:
df =train
df.apply(lambda x: pd.lib.infer_dtype(x.values))

train_id              integer
name                  unicode
item_condition_id     integer
category_name         unicode
brand_name            unicode
price                floating
shipping              integer
item_description      unicode
dtype: object

In [347]:
def fill_missing_values(df):
    df.category_name.fillna(value="Other", inplace=True)
    df.brand_name.fillna(value="missing", inplace=True)
    df.item_description.fillna(value="None", inplace=True)
    return df

train = fill_missing_values(train)
test = fill_missing_values(test)

In [348]:
def get_cats(merge):
    merge['general_cat_1'],merge['general_cat_2'], merge['subcat_1'],merge['subcat_1_2'], merge['subcat_2'],merge['subcat_2_2'] = \
    zip(*merge['category_name'].apply(lambda x: split_cat(x)))
    merge.drop('category_name', axis=1, inplace=True)
    return merge

all_sets = [train,test]
all_sets = map(get_cats,all_sets)

In [349]:
train.head(100)

,train_id,name,item_condition_id,brand_name,price,shipping,item_description,general_cat_1,general_cat_2,subcat_1,subcat_1_2,subcat_2,subcat_2_2
0,0,MLB Cincinnati Reds T Shirt Size XL,3,,2.397895,1,No description yet,Men,no_label,Tops,no_label,T-shirts,no_label
1,1,Razer BlackWidow Chroma Keyboard,3,Razer,3.970292,0,This keyboard is in great condition and works ...,Electronics,no_label,Computers,Tablets,Components,Parts
2,2,AVA-VIV Blouse,1,Target,2.397895,1,Adorable top with a hint of lace and a key hol...,Women,no_label,Tops,Blouses,Blouse,no_label
3,3,Leather Horse Statues,1,,3.583519,1,New with tags. Leather horses. Retail for [rm]...,Home,no_label,Home Décor,no_label,Home Décor Accents,no_label
4,4,24K GOLD plated rose,1,,3.806662,0,Complete with certificate of authenticity,Women,no_label,Jewelry,no_label,Necklaces,no_label
5,5,Bundled items requested for Ruie,3,,4.094345,0,"Banana republic bottoms, Candies skirt with ma...",Women,no_label,Other,no_label,Other,no_label
6,6,Acacia pacific tides santorini top,3,Acacia Swimwear,4.174387,0,Size small but straps slightly shortened to fi...,Women,no_label,Swimwear,no_label,Two-Piece,no_label
7,7,Girls cheer and tumbling bundle of 7,3,Soffe,1.945910,1,You get three pairs of Sophie cheer shorts siz...,Sports,Outdoors,Apparel,no_label,Girls,no_label
8,8,Girls Nike Pro shorts,3,Nike,2.995732,0,Girls Size small Plus green. Three shorts total.,Sports,Outdoors,Apparel,no_label,Girls,no_label
9,9,Porcelain clown doll checker pants VTG,3,,2.197225,0,I realized his pants are on backwards after th...,Vintage,Collectibles,Collectibles,no_label,Doll,no_label


In [350]:
train['order_id'] = train['train_id']
for grp_cols in [['item_condition_id','brand_name'],['item_condition_id','subcat_2'],['item_condition_id','subcat_1'],
                 ['subcat_2','shipping'],['brand_name','shipping'],['brand_name','subcat_2'],['brand_name','subcat_1'],
                 ['brand_name','subcat_1_2']]:
    group = get_stats_target(train,grp_cols,['price'],drop_count=False,filter_count=0,prefix="agg").reset_index()
    train = train.merge(group,on=grp_cols,how='left')
    test = test.merge(group,on=grp_cols,how='left')
    

prefix is agg,group column is item_condition_id#brand_name,agg total count is 11352; after filter , agg total count is 11352, filter ratio is 8.80901267708e-05 %
prefix is agg,group column is item_condition_id#subcat_2,agg total count is 3150; after filter , agg total count is 3150, filter ratio is 0.000317459309662 %
prefix is agg,group column is item_condition_id#subcat_1,agg total count is 527; after filter , agg total count is 527, filter ratio is 0.00189749720119 %
prefix is agg,group column is subcat_2#shipping,agg total count is 1654; after filter , agg total count is 1654, filter ratio is 0.000604591266073 %
prefix is agg,group column is brand_name#shipping,agg total count is 7772; after filter , agg total count is 7772, filter ratio is 0.000128666844235 %
prefix is agg,group column is brand_name#subcat_2,agg total count is 36915; after filter , agg total count is 36915, filter ratio is 2.70892517751e-05 %
prefix is agg,group column is brand_name#subcat_1,agg total count is 182

In [351]:
# train.columns
for t,name in zip([x.name for x in train.dtypes],train.columns):
    if t == 'object':
        train[name] = train[name].astype('category')
        test[name] = test[name].astype('category')

In [352]:
train.dtypes

train_id                                                    int64
name                                                     category
item_condition_id                                           int64
brand_name                                               category
price                                                     float64
shipping                                                    int64
item_description                                         category
general_cat_1                                            category
general_cat_2                                            category
subcat_1                                                 category
subcat_1_2                                               category
subcat_2                                                 category
subcat_2_2                                               category
order_id                                                    int64
index_x                                                     int64
agg__price

In [356]:
grp_features = []
grp_features = grp_features + [x for x in train.columns if "by" in x ]
print grp_features

['agg__price__mean_by__item_condition_id#brand_name', 'agg__price__median_by__item_condition_id#brand_name', 'agg__price__max_by__item_condition_id#brand_name', 'agg__price__min_by__item_condition_id#brand_name', 'agg__price__std_by__item_condition_id#brand_name', 'agg__price__skew_by__item_condition_id#brand_name', 'agg__price__sem_by__item_condition_id#brand_name', 'agg__price__var_by__item_condition_id#brand_name', 'agg__order_id__count_by__item_condition_id#brand_name', 'agg__price__mean_by__item_condition_id#subcat_2', 'agg__price__median_by__item_condition_id#subcat_2', 'agg__price__max_by__item_condition_id#subcat_2', 'agg__price__min_by__item_condition_id#subcat_2', 'agg__price__std_by__item_condition_id#subcat_2', 'agg__price__skew_by__item_condition_id#subcat_2', 'agg__price__sem_by__item_condition_id#subcat_2', 'agg__price__var_by__item_condition_id#subcat_2', 'agg__order_id__count_by__item_condition_id#subcat_2', 'agg__price__mean_by__item_condition_id#subcat_1', 'agg__pric

In [364]:
basic_features = [
#     u'name', 
    u'item_condition_id',
       u'brand_name', 
                  u'shipping',
                  u'general_cat_1',
                  u'general_cat_2',
                  u'subcat_1',
                  u'subcat_1_2',
                  u'subcat_2', 
                  u'subcat_2_2'
                 ]

In [368]:
develop=True
params = {
    'learning_rate': 0.3,
    'application': 'regression',
    'max_depth': 50,
    'num_leaves': 2**8-1,
    'verbosity': -1,
    'metric': 'RMSE',
    'data_random_seed': 1,
    'booster': 'gbtree',
#     'bagging_fraction': 0.8,
#     'bagging_freq': 5,
    'feature_fraction': 0.5,
    'nthread': 4,
    'min_data_in_leaf': 50,
#     'max_bin': 31
}
#base line -- 4778
features = basic_features + grp_features

label = ['price']

# X = train[features]
# y = 
X, y = train[features],train[label].values.astype(np.float32).flatten()

if develop:
    train_X, valid_X, train_y, valid_y = train_test_split(
        X, y,train_size=0.3,test_size=0.05, random_state=100)

d_train = lgb.Dataset(train_X, label=train_y)
watchlist = [d_train]
if develop:
    d_valid = lgb.Dataset(valid_X, label=valid_y)
    watchlist = [d_train, d_valid]

model = lgb.train(params, train_set=d_train, num_boost_round=1000, valid_sets=watchlist,
                  early_stopping_rounds=30, verbose_eval=30)

Training until validation scores don't improve for 30 rounds.
[30]	training's rmse: 0.526588	valid_1's rmse: 0.543661
Early stopping, best iteration is:
[13]	training's rmse: 0.531901	valid_1's rmse: 0.542582


In [366]:
gbm =model
df = pd.DataFrame({'feature': gbm.feature_name(), 'importances_split': gbm.feature_importance(),'importances_gain': gbm.feature_importance('gain')})
df['fscore_split'] = df['importances_split'] / df['importances_split'].sum()
df['fscore_gain'] = df['importances_gain'] / df['importances_gain'].sum()

    # print(df)
print(df.sort_values('fscore_split', ascending=False).to_string(index=False))
    

feature  importances_gain  importances_split  fscore_split  fscore_gain
                                       brand_name       2352.166839                222      0.154167     0.014360
                                         subcat_2       4197.028383                191      0.132639     0.025623
                                         subcat_1       1599.085514                 75      0.052083     0.009762
         agg__price__mean_by__brand_name#subcat_2     129242.360591                 58      0.040278     0.789014
          agg__price__std_by__brand_name#subcat_2        439.083305                 37      0.025694     0.002681
          agg__price__var_by__brand_name#subcat_2        115.361861                 32      0.022222     0.000704
                                item_condition_id       3492.611331                 29      0.020139     0.021322
agg__price__skew_by__item_condition_id#brand_name        178.490630                 28      0.019444     0.001090
          agg__p

In [369]:
train.head()

,train_id,name,item_condition_id,brand_name,price,shipping,item_description,general_cat_1,general_cat_2,subcat_1,...,index_y,agg__price__mean_by__brand_name#subcat_1_2,agg__price__median_by__brand_name#subcat_1_2,agg__price__max_by__brand_name#subcat_1_2,agg__price__min_by__brand_name#subcat_1_2,agg__price__std_by__brand_name#subcat_1_2,agg__price__skew_by__brand_name#subcat_1_2,agg__price__sem_by__brand_name#subcat_1_2,agg__price__var_by__brand_name#subcat_1_2,agg__order_id__count_by__brand_name#subcat_1_2
0,0,MLB Cincinnati Reds T Shirt Size XL,3,,2.397895,1,No description yet,Men,no_label,Tops,...,19,2.814663,2.772589,7.601402,0.000000,0.709608,0.565030,0.000999,0.503544,504901
1,1,Razer BlackWidow Chroma Keyboard,3,Razer,3.970292,0,This keyboard is in great condition and works ...,Electronics,no_label,Computers,...,6415,3.717555,3.783931,4.454347,2.833213,0.457251,-0.164932,0.083482,0.209079,30
2,2,AVA-VIV Blouse,1,Target,2.397895,1,Adorable top with a hint of lace and a key hol...,Women,no_label,Tops,...,7477,2.419164,2.397895,3.610918,1.386294,0.367542,0.300143,0.028527,0.135087,166
3,3,Leather Horse Statues,1,,3.583519,1,New with tags. Leather horses. Retail for [rm]...,Home,no_label,Home Décor,...,19,2.814663,2.772589,7.601402,0.000000,0.709608,0.565030,0.000999,0.503544,504901
4,4,24K GOLD plated rose,1,,3.806662,0,Complete with certificate of authenticity,Women,no_label,Jewelry,...,19,2.814663,2.772589,7.601402,0.000000,0.709608,0.565030,0.000999,0.503544,504901


# base func

In [105]:
def get_stats_target(df, group_column, target_column_list, tgt_stats=['mean', 'median', 'max', 'min', 'std','skew','sem','var'],
                     prefix="hehe", drop_count=True, drop_raw_col=True, filter_count=10):
    df_old = df.copy()
    grouped = df_old.groupby(group_column)
    grouped.order_id.count()
    rs = []
    for target_column in target_column_list:
        def gene_col_names(stat):
            return '{}__{}__{}_by__{}'.format(prefix, target_column, stat, "#".join(group_column))

        the_stats = grouped[target_column].agg(tgt_stats).reset_index()
        # print the_stats.columns
        the_stats.columns = group_column + map(gene_col_names, tgt_stats)
        if drop_raw_col:
            the_stats.drop(group_column, axis=1, inplace=True)
        rs.append(the_stats)

    count = grouped["order_id"].agg(['count']).reset_index()
    count_col_name = '{}__{}__{}_by__{}'.format(prefix, "order_id", "count", "#".join(group_column))
    count.columns = group_column + [count_col_name]
    agg_rs = pd.concat(rs + [count], axis=1)
    af_flt = agg_rs[agg_rs[count_col_name] >= filter_count]

    print "prefix is {},group column is {},agg total count is {}; after filter , agg total count is {}, filter ratio is {} %". \
        format(prefix, "#".join(group_column), len(agg_rs), len(af_flt),
               (1 - len(af_flt) / (len(agg_rs) + 0.01)) * 100)

    if drop_count:
        return af_flt[[x for x in af_flt.columns if "count" not in x]]
    else:
        return af_flt

In [171]:
import itertools as it 
def fill_2(x):
    if len(x) ==1:
        x+=['no_label']
    return x
def split_cat(text):
    try:
        rs = list(it.chain(*map(lambda x:fill_2(x.split("&")),text.split("/")[:3])))
        if len(rs) ==4:
            rs+=["no label"]*2
        if len(rs) ==2:
            rs+=["no label"]*4
    except :
        print text
        return ["no label"]*6
#     if len(rs) == 0:
#         rs+=["no label","no label","no label"]
    return rs

In [163]:
train['category_name'].spl

,train_id,name,item_condition_id,brand_name,price,shipping,item_description,general_cat,subcat_1,subcat_2
0,0,MLB Cincinnati Reds T Shirt Size XL,3,missing,2.397895,1,No description yet,Men,Tops,T-shirts
1,1,Razer BlackWidow Chroma Keyboard,3,Razer,3.970292,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts
2,2,AVA-VIV Blouse,1,Target,2.397895,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse
3,3,Leather Horse Statues,1,missing,3.583519,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents
4,4,24K GOLD plated rose,1,missing,3.806662,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces
